Pronto para rodar no JupyterLab

# Processamento e Modelagem de Dados

Neste notebook, vamos realizar as seguintes etapas:

1. Conectar ao banco de dados MySQL.
2. Fazer o download de um CSV armazenado no S3.
3. Inserir o DataFrame no MySQL.
4. Modificar o DataFrame e criar uma nova tabela.
5. Preparar os dados para treinamento.
6. Treinar um modelo RandomForestClassifier.
7. Avaliar o modelo e salvar no S3.

## Importação de Bibliotecas


In [55]:
!pip install pymysql
import pandas as pd
from io import BytesIO, StringIO
import boto3
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sqlalchemy import create_engine
import requests


## Configurações e Conexão ao Banco de Dados


In [56]:
endpoint = 'database-grupo5.czycoa0q690g.us-east-1.rds.amazonaws.com'
port = 3306
username = 'admin'
password = 'fysren-ximruH-2hoqjy'
dbname = 'hotel_reservations_db'

print("Conectando ao banco de dados...")
# Conectar ao banco de dados
engine = create_engine(f'mysql+pymysql://{username}:{password}@{endpoint}:{port}/{dbname}')
print("Conexão com o banco de dados estabelecida.")


Conectando ao banco de dados...
Conexão com o banco de dados estabelecida.


## Download do CSV do S3


In [57]:
# URL do CSV no S3
csv_url = 'https://hotelreservationsgrupo5.s3.us-east-1.amazonaws.com/hotel_reservations.csv'
print(f"Fazendo o download do CSV de {csv_url}...")

# Fazer o download do CSV
response = requests.get(csv_url)
response.raise_for_status()  # Verifica se houve erro no download
print("Download do CSV concluído.")

# Ler o CSV para um DataFrame do Pandas
csv_data = StringIO(response.text)
df = pd.read_csv(csv_data)
print("CSV lido com sucesso.")


Fazendo o download do CSV de https://hotelreservationsgrupo5.s3.us-east-1.amazonaws.com/hotel_reservations.csv...
Download do CSV concluído.
CSV lido com sucesso.


## Inserir Dados no Banco de Dados


In [58]:
print("Inserindo dados no banco de dados...")
df.to_sql('hotel_reservations', con=engine, if_exists='replace', index=False)
print("Dados inseridos no banco de dados.")
df

Inserindo dados no banco de dados...
Dados inseridos no banco de dados.


,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,INN36271,3,0,2,6,Meal Plan 1,0,Room_Type 4,85,2018,8,3,Online,0,0,0,167.80,1,Not_Canceled
36271,INN36272,2,0,1,3,Meal Plan 1,0,Room_Type 1,228,2018,10,17,Online,0,0,0,90.95,2,Canceled
36272,INN36273,2,0,2,6,Meal Plan 1,0,Room_Type 1,148,2018,7,1,Online,0,0,0,98.39,2,Not_Canceled
36273,INN36274,2,0,0,3,Not Selected,0,Room_Type 1,63,2018,4,21,Online,0,0,0,94.50,0,Canceled


## Modificar o DataFrame
Vamos adicionar uma nova coluna `label_avg_price_per_room`, remover a coluna `avg_price_per_room`, e criar uma nova tabela com as modificações.


In [59]:
# Adicionar a coluna label_avg_price_per_room
print("Adicionando a coluna 'label_avg_price_per_room'...")
df['label_avg_price_per_room'] = df['avg_price_per_room'].apply(lambda x: 1 if x <= 85 else (2 if x < 115 else 3))

# Remover a coluna avg_price_per_room
print("Removendo a coluna 'avg_price_per_room'...")
df.drop(columns=['avg_price_per_room'], inplace=True)
df.drop(columns=['Booking_ID'], inplace=True)

# Criar uma nova tabela com o DataFrame modificado
print("Criando a tabela 'hotel_reservations_updated' com os dados modificados...")
df.to_sql('hotel_reservations_updated', con=engine, if_exists='replace', index=False)
print("Tabela 'hotel_reservations_updated' criada com sucesso.")
df


Adicionando a coluna 'label_avg_price_per_room'...
Removendo a coluna 'avg_price_per_room'...
Criando a tabela 'hotel_reservations_updated' com os dados modificados...
Tabela 'hotel_reservations_updated' criada com sucesso.


,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,no_of_special_requests,booking_status,label_avg_price_per_room
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,0,Not_Canceled,1
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,1,Not_Canceled,2
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,0,Canceled,1
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,0,Canceled,2
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,0,Canceled,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,INN36271,3,0,2,6,Meal Plan 1,0,Room_Type 4,85,2018,8,3,Online,0,0,0,1,Not_Canceled,3
36271,INN36272,2,0,1,3,Meal Plan 1,0,Room_Type 1,228,2018,10,17,Online,0,0,0,2,Canceled,2
36272,INN36273,2,0,2,6,Meal Plan 1,0,Room_Type 1,148,2018,7,1,Online,0,0,0,2,Not_Canceled,2
36273,INN36274,2,0,0,3,Not Selected,0,Room_Type 1,63,2018,4,21,Online,0,0,0,0,Canceled,2


## Preparar os Dados para Treinamento
Vamos preparar os dados para o treinamento do modelo e realizar a codificação das variáveis categóricas.


In [60]:
if 'label_avg_price_per_room' in df.columns:
    print("Preparando os dados para treinamento...")
    X = df.drop(['label_avg_price_per_room'], axis=1)
    y = df['label_avg_price_per_room']

    # Label encoding para colunas categóricas
    print("Realizando o label encoding para colunas categóricas...")
    categorical_cols = X.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        X[col] = X[col].astype('category').cat.codes

    # Dividir os dados em treino e teste
    print("Dividindo os dados em treino e teste...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Preparando os dados para treinamento...
Realizando o label encoding para colunas categóricas...
Dividindo os dados em treino e teste...


## Treinamento e Avaliação do Modelo
Vamos treinar o modelo `RandomForestClassifier` e avaliar seu desempenho.


In [61]:
    # Treinar o modelo
    print("Treinando o modelo RandomForestClassifier...")
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    print("Modelo treinado com sucesso.")

    # Avaliar o modelo
    print("Avaliando o modelo...")
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f'Acurácia: {accuracy}')


Treinando o modelo RandomForestClassifier...
Modelo treinado com sucesso.
Avaliando o modelo...
Acurácia: 0.8505858028945554


## Salvar o Modelo no S3
Vamos salvar o modelo treinado em um bucket do S3.


In [63]:
    # Salvar o modelo em um buffer de memória usando BytesIO
    print("Salvando o modelo em um buffer de memória...")
    model_buffer = BytesIO()
    joblib.dump(model, model_buffer)
    model_buffer.seek(0)  # Voltar ao início do buffer

    # Conectar ao S3 e enviar o modelo
    print("Conectando ao S3...")
    s3 = boto3.client('s3')
    bucket_name = 'hotelreservationsgrupo5'
    model_file_name = 'random_forest_model.pkl'

    print(f"Enviando o modelo para o bucket {bucket_name} com o nome {model_file_name}...")
    s3.put_object(Bucket=bucket_name, Key=model_file_name, Body=model_buffer)
    print(f"Modelo salvo no bucket {bucket_name} com o nome {model_file_name}.")


Salvando o modelo em um buffer de memória...
Conectando ao S3...
Enviando o modelo para o bucket hotelreservationsgrupo5 com o nome random_forest_model.pkl...
Modelo salvo no bucket hotelreservationsgrupo5 com o nome random_forest_model.pkl.
